# Interacting with MODFLOW-API Interface objects

The purpose of this notebook is to show the MODFLOW-API interface objects and introduce the user to the data types and how to interact with the objects. 

**Note**: This notebook does not show how to run a model using the modflow-api. Instead it is an illustration of how to access and work with the data types that are returned to a user defined callback function. For an example of how to run a custom modflow-api model run, see the notebook titled "xxxx.ipynb".

In [ ]:
import modflowapi
from modflowapi.interface import Simulation
from pathlib import Path
import platform

Define the paths to the model and the Modflow shared library

In [ ]:
sim_ws = Path("../test_model")
dll = "./libmf6"
if platform.system().lower() == "windows":
    ext = ".dll"
elif platform.system().lower() == "linux":
    ext = ".so"
else:
    ext = ".dylib"
    
dll = Path(dll + ext)

#### Initializing the API model object

The modflow api allows users to initialize an object that can be used to interact with the model. This processes is done automatically with the `modflowapi.run_model` function call. We're going to initialize an object outside of that call as a demonstration of the interface data objects

In [ ]:
mf6 = modflowapi.ModflowApi(dll, working_directory=sim_ws)
mf6.initialize()

# let's advance the model to the first timestep
dt = mf6.get_time_step()
mf6.prepare_time_step(dt)

## The `Simulation` object 

The simulation object is the top level container for the modflowapi interface classes. This container holds methods and other objects that allow the user to access boundary condition pointer data without assembling the specific memory addresses of the modflow data. 

Let's take a look at the `Simulation` object

In [ ]:
sim = Simulation.load(mf6)
sim

The simulation object allows the user to access models by name and has a number of handy properties

In [ ]:
mnames = sim.model_names
mnames

In [ ]:
kstp, kper = sim.kstp, sim.kper
kstp, kper

In [ ]:
nstp = sim.nstp
nstp

## The `Model` object

`Model` objects are accessed from the `Simulation` object and are a container for packages. These objects allow the user to view which packages are available and access those packages. 

The following cells show the main attributes and functions available on the `Model` object

Model objects are accessible through the `get_model` function and as attributes on the sim object

In [ ]:
model = sim.get_model('test_model')
model

In [ ]:
# approach 2
model = sim.test_model
model

There are also a number of other functions available including the following:

In [ ]:
model.shape

In [ ]:
model.size

In [ ]:
model.solution_id

A list of all package names that are accesible is also available

In [ ]:
model.package_names

## The `Package` object(s)

Each package is contained in `Package` container. There are three types depending on the input data. We'll access and take a look at each of the types of `Package` containers.

Packages can be accessed from the `Model` object using `get_package()` or by attribute

In [ ]:
# example 1: get a package using get_package
rch = model.get_package("rcha_0")
rch

In [ ]:
# example 2: get a package by package name attribute
wel = model.wel
wel

In [ ]:
# example 3: get all packages based on a package type
rch_pkgs = model.rch
rch_pkgs

### `ListPackage` objects

`ListPackage` objects are the primary object type of stress period data. The exception to this rule is the advanced packages which will be discussed later. 

`ListPackage` objects allow users to access stress period data as a numpy recarray or as a pandas dataframe.

In [ ]:
recarray = rch.stress_period_data.values
recarray[0:10]

In [ ]:
df = rch.stress_period_data.dataframe
df.head()

### Updating values for `ListPackage` based data

There are multiple ways to update values for `ListPackage` based data. The `.values` and `.dataframe` attributes can be used, or the object can be directly indexed if the user knows the underlying data. Here are some examples

In [ ]:
recarray['recharge'][0] *= 100
rch.stress_period_data.values = recarray

# show that values have been updated
recarray = rch.stress_period_data.values
recarray[0:5]

In [ ]:
df = rch.stress_period_data.dataframe
df.loc[1, "recharge"] = 10000
rch.stress_period_data.dataframe = df

# show that values have been updated
df = rch.stress_period_data.dataframe
df.head()

#### Interfacing directly with the `.stress_period_data` attribute

The `.stress_period_data` attribute returns a container class that interacts with the internal modflow pointers. The data can be adjusted by interacting with `.stress_period_data` in the same fashion as changing data in a numpy recarray. 

In [ ]:
rch.stress_period_data["recharge"] *= 100

In [ ]:
df = rch.stress_period_data.dataframe
df.head()

### `ArrayPackage` objects

The `ArrayPackage` class is used as a container for packages such as `DIS`, `NPF`, and `IC` that do not contain any sort of stress period data. These packages are used primarily to define model connectivity, initial conditions, and hydraulic parameters of the basin. 

In [ ]:
npf = model.npf
print(npf.rhs)

For an `ArrayPackage` type object, variable names can be viewed by calling the `.variable_names` property

In [ ]:
npf.variable_names

### Updating values for `ArrayPackage` objects

Two methods are available for accessing and updating data in `ArrayPackage` objects. `get_array()` and `set_array()` methods can be used to get and set data. Arrays can also be accessed as attributes on the object.

Using `get_array()` and `set_array()`

In [ ]:
hk = npf.get_array("k11")
hk

In [ ]:
hk[0, 0:5, 0:5] = 50
npf.set_array("k11", hk)

In [ ]:
# confirm that the data has been updated
hk = npf.get_array("k11")
hk

# what we cant do:
npf.k11[:, 0:5, 0:5] = 1000
x = npf.k11
x[:, 0:5, 0:5] = 1000

npf.k11

Getting and setting data by attribute

In [ ]:
# needs an update for inplace operations....
k33 = npf.k33
k33[0, 0:5, 0:5] = 50
npf.k33 = k33

In [ ]:
# confirm that the data has been updated
npf.k33

## Accessing "advanced variables"

Advanced variables in this context are variables that would not normally need to be accessed by the user, and in many cases changes to these variables would cause the Modflow simulation to do unexpected things. 

For each package object a list of avanced variables can be returned by calling the `advanced_vars` attribute

In [ ]:
wel = model.wel_0
wel.advanced_vars

The user can access and change these values, _at their own risk_, using the `.get_advanced_var()` and `.set_advanced_var()` methods. Data is returned to the user in the internal modflowapi structure. 

In [ ]:
wel.get_advanced_var("ibcnum")

### Advanced Packages

Certain packages only support accessing data through the `.get_advanced_var()` and `.set_advanced_var()` methods. These packages, sometime refered to as "advanced packages" include, BUY, CSUB, GNC, HFB, MAW, MVR, SFR, and UZF. 

-------

Let's close the existing modflowapi shared library object and look at an example of how this is all used in practice

In [ ]:
mf6.finalize()

# Putting it all together and running a modflowapi simulation

To run a simulation using the built in modflowapi runner the user needs to create a function that will receive callbacks at different steps in the simulation run. For the remainder of this notebook, we'll show how to create a callback function and use it with the `modflowapi.run_model()` method.

## Create a callback function for adjusting model data

The callback function allows users to wrap function that updates the modflow model at different steps. The `modflowapi.Callbacks` object allows users to find the particular solution step that they are currently in. `modflowapi.Callbacks` includes:

   - `Callbacks.initalize`: the initialize callback sends loaded simulation data back to the user to make adjustments before the simulation| begins solving. This callback only occurs once at the beginning of the MODFLOW6 simulation
   - `Callbacks.stress_period`: the stress period callback sends simulation data for each solution group to the user to make adjustments to stress packages at the beginning of each stress period.
   - `Callbacks.timestep_start`: the timestep_start callback sends simulation data for each solution group to the user to make adjustments to stress packages at the beginning of each timestep.
   - `Callbacks.timestep_end`: ?????
   - `Callbacks.iteration_start`: the iteration_start callback sends simulation data for each solution group to the user to make adjustments to stress packages at the beginning of each outer solution iteration.
   - `Callbacks.iteration_end`: the iteration_end callback sends simulation data for each solution group to the user to make adjustments to stress packages and check values of stress packages at the end of each outer solution iteration.
   
The user can use any or all of these callbacks within their callback function

In [ ]:
from modflowapi import Callbacks

In [ ]:
def callback_function(sim, step):
    """
    A demonstration function that dynamically adjusts recharge
    and pumping in a modflow-6 model through the MODFLOW-API
    
    Parameters
    ----------
    sim : modflowapi.Simulation
        A simulation object for the solution group that is 
        currently being solved
    step : enumeration
        modflowapi.Callbacks enumeration object that indicates
        the part of the solution modflow is currently in.
    """
    ml = sim.test_model
    if step == Callbacks.initialize:
        print(sim.models)
    
    if step == Callbacks.stress_period:
        # adjust recharge for stress periods 7 through 12
        if sim.kper <= 6:
            rcha = ml.rcha_0
            spd = rcha.stress_period_data
            print(f"updating recharge: stress_period={ml.kper}")
            spd["recharge"] += 0.40 * sim.kper
        
    
    if step == Callbacks.timestep_start:
        print(f"updating wel flux: stress_period={ml.kper}, timestep={ml.kstp}")
        ml.wel.stress_period_data["flux"] -= ml.kstp * 1.5
    
    if step == Callbacks.iteration_start:
        # we can implement complex solutions to boundary conditions here!
        pass
    

The callback function is then passed to `modflowapi.run_model`

In [ ]:
modflowapi.run_model(dll, sim_ws, callback_function, verbose=False)